In [312]:
from __future__ import annotations
import math
from utils import Utils, AOC
from session import SESSION
from collections import defaultdict
from functools import reduce
import numpy as np
import re

In [2]:
aoc = AOC(session=SESSION)

In [3]:
aoc.verify_session()

True

In [4]:
aoc.get_today_file().analyse().head()

Data correctly downloaded and saved locally for next usage.
100% of data are digits. Analyse as numbers.
0 empty line(s) found. Analyse as monline data.
===== HEAD (5/100) =====
[9 2 2 9 0 1 2 3 0]
[2 0 2 6 4 7 3 0 3 0 0 6]
[7 2 2 9 6 5 2 4 5]
[7 8 2 1 5 4 2 9 7 8]
[0 7 1 6 6 0 7 9 4]


/home/bhasher/Documents/development/projects/personal/ProgrammingChallenges/utils_components/aoc2.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array([np.array([parse_number(el) for el in re.findall(r'\d+\.\d+|\d+', line)]) for line in group])


In [397]:
class Node:
    def __init__(self, value: int, previous=None, next=None):
        self.value: int = int(value)
        self.previous: Node = previous
        self.next: Node = next

    def __repr__(self):
        return str(self.value)

    def __copy__(self):
        return Node(self.value, self.previous, self.next)

In [328]:
class SnNumber:
    def __init__(self, left_child, right_child, parent=None):
        self.left_child: SnNumber|Node = left_child
        self.right_child: SnNumber|Node = right_child
        self.parent: SnNumber = parent
        self.head = None

    @staticmethod
    def parse(text: str, parent=None):
        text = text.strip().replace(' ', '')
        opening = 0
        for i, el in enumerate(text):
            if el == '[':
                opening += 1
            elif el == ']':
                opening -= 1
            elif el == ',' and opening == 1:
                snn = SnNumber(None, None, parent=parent)

                snn.left_child = Node(left) if (left := text[1:i]).isdigit() else SnNumber.parse(left, snn)
                snn.right_child = Node(right) if (right := text[i+1:-1]).isdigit() else SnNumber.parse(right, snn)

                return snn


    def set_sub_links(self):
        lst = list(self.deep_iter())

        self.head = lst[0]

        for i in range(1,len(lst)):
            lst[i-1].next = lst[i]
            lst[i].previous = lst[i-1]

    def __repr__(self):
        return f'[{self.left_child},{self.right_child}]'

    def deep_iter(self):
        for child in self:
            if isinstance(child, SnNumber):
                for sub_child in child.deep_iter():
                    yield sub_child
            else:
                yield child


    def __iter__(self):
        yield self.left_child
        yield self.right_child

    def __add__(self, other):
        snn = SnNumber(self, other)
        self.parent = snn
        other.parent = snn

        snn.set_sub_links()
        while snn.try_explode() or snn.try_split():
            pass

        return snn

    def try_split(self) -> bool:
        for i, child in enumerate(self):
            if isinstance(child, SnNumber):
                if child.try_split():
                    return True
            else:
                if child.value >= 10:
                    left = Node(math.floor(child.value/2), child.previous, None)
                    right = Node(math.ceil(child.value/2), left, child.next)
                    if child.previous:
                        child.previous.next = left
                    if child.next:
                        child.next.previous = right
                    left.next = right

                    if i == 0:
                        self.left_child = SnNumber(left, right, self)
                    else:
                        self.right_child = SnNumber(left, right, self)

                    return True
        return False

    def try_explode(self, deep=1) -> bool:
        if deep <= 4:
            return self and (isinstance(self.left_child, SnNumber) and self.left_child.try_explode(deep+1)
                   or isinstance(self.right_child, SnNumber) and self.right_child.try_explode(deep+1))

        new_node = Node(0, self.left_child.previous, self.right_child.next)

        #print(self, self.parent)

        if self.left_child.previous:
            self.left_child.previous.value += self.left_child.value
            self.left_child.previous.next = new_node
        if self.right_child.next:
            self.right_child.next.value += self.right_child.value
            self.right_child.next.previous = new_node

        if self.parent.left_child is self:
            self.parent.left_child = new_node
        else:
            self.parent.right_child = new_node

        return True

    def magnitude(self):
        subs = [child.magnitude() if isinstance(child, SnNumber) else child.value for child in self]
        return subs[0] * 3 + subs[1] * 2

    def __copy__(self):
        return SnNumber(
            self.left_child.__copy__(),
            self.left_child.__copy__()
        )

In [361]:
data= """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

In [419]:
data = aoc.raw

In [420]:
snns = [SnNumber.parse(line) for line in data.split('\n')]

In [399]:
result = reduce(lambda a,b: a+b, snns)

In [401]:
result.magnitude()

4140

In [423]:
magns = []
for i, snn1 in enumerate(snns):
    print(i, end=' ')
    for j, snn2 in enumerate(snns):
        snns = [SnNumber.parse(line) for line in data.split('\n')]
        if i != j:
            magns.append(((snns[i] + snns[j]).magnitude(), i, j))
            #magns.append(((snns[j] + snns[i]).magnitude(), i, j))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [424]:
# > 2963

In [425]:
max(magns)

(4669, 31, 83)